# <img src="https://lh3.googleusercontent.com/mUTbNK32c_DTSNrhqETT5aQJYFKok2HB1G2nk2MZHvG5bSs0v_lmDm_ArW7rgd6SDGHXo0Ak2uFFU96X6Xd0GQ=w160-h128" width="45" valign="top" alt="BigQuery"> Synthetic Data Generation Using LLMs

## Initialize Python

In [ ]:
import pandas as pd
import json
import bigframes.pandas as bf
#from bigframesllm import BigFramesLLM
from bigframes.ml.llm import PaLM2TextGenerator

In [ ]:
from google.cloud import bigquery
client = bigquery.Client()

In [ ]:
PROJECT_ID = "data-analytics-preview"
REGION = "us"
DATASET_ID = "coffee_curated"
CONNECTION_NAME = "vertex-ai"

connection = f"{PROJECT_ID}.{REGION}.{CONNECTION_NAME}"

In [ ]:
# bf.reset_session() # if you need to change the region
bf.options.bigquery.project = PROJECT_ID
bf.options.bigquery.location = REGION

In [ ]:
session = bf.get_global_session()

llm_model = PaLM2TextGenerator(session=session, connection_name=connection)

## Supporting Functions

In [ ]:
def PrettyPrintJson(json_string):
  json_object = json.loads(json_string)
  json_formatted_str = json.dumps(json_object, indent=2)
  print(json_formatted_str)
  return json.dumps(json_object)

In [ ]:
def LLM(prompt, isOutputJson, max_output_tokens=1024, temperature=0, top_p=0, top_k=1):
  print()
  print("Prompt: ", prompt)
  print()
  df_prompt = pd.DataFrame(
          {
              "prompt": [prompt],
          })
  bf_df_prompt = bf.read_pandas(df_prompt)
  prediction = llm_model.predict(bf_df_prompt,
                                 max_output_tokens=max_output_tokens,
                                 temperature=temperature, # 0 to 1 (1 random)
                                 top_p=top_p, # 0 to 1 (1 random)
                                 top_k=top_k, # (1 to 40 random)
                                 ).to_pandas()
  try:
    # Remove common LLM output mistakes
    result = prediction['ml_generate_text_llm_result'][0]

    result = result.replace("```json\n","")
    result = result.replace("```JSON\n","")
    result = result.replace("```json","")
    result = result.replace("```JSON","")
    result = result.replace("```sql\n","")
    result = result.replace("```SQL\n","")
    result = result.replace("```sql","")
    result = result.replace("```SQL","")
    result = result.replace("```","")

    if isOutputJson:
      json_string = PrettyPrintJson(result)
      json_string = json_string.replace("'","\\'")
      json_string = json_string.strip()
      return json_string
    else:
      result = result.replace("'","\\'")
      result = result.strip()
      return result

  except:
    print("Error (raw): ", prediction['ml_generate_text_llm_result'][0])
    print("Error (result): ", result)


In [ ]:
def GetTableSchema(dataset_name, table_name):
  import io

  dataset_ref = client.dataset(dataset_name, project=PROJECT_ID)
  table_ref = dataset_ref.table(table_name)
  table = client.get_table(table_ref)

  f = io.StringIO("")
  client.schema_to_json(table.schema, f)
  return f.getvalue()

In [ ]:
def GetForeignKeys(dataset_name, table_name, field_name):
  sql = f"""
  SELECT STRING_AGG(CAST({field_name} AS STRING), "," ORDER BY {field_name}) AS result
    FROM `{PROJECT_ID}.{dataset_name}.{table_name}`
  """
  #print(sql)
  df_result = client.query(sql).to_dataframe()
  #display(df_result)
  return df_result['result'].iloc[0]

In [ ]:
def GetDistinctValues(dataset_name, table_name, field_name):
  sql = f"""
  SELECT STRING_AGG(DISTINCT {field_name}, "," ) AS result
    FROM `{PROJECT_ID}.{dataset_name}.{table_name}`
  """
  #print(sql)
  df_result = client.query(sql).to_dataframe()
  #display(df_result)
  return df_result['result'].iloc[0]

In [ ]:
def GetStartingValue(dataset_name, table_name, field_name):
  sql = f"""
  SELECT IFNULL(MAX({field_name}),0) + 1 AS result
    FROM `{PROJECT_ID}.{dataset_name}.{table_name}`
  """
  #print(sql)
  df_result = client.query(sql).to_dataframe()
  #display(df_result)
  return df_result['result'].iloc[0]

In [ ]:
def GetMaximumValue(dataset_name, table_name, field_name):
  sql = f"""
  SELECT IFNULL(MAX({field_name}),0) AS result
    FROM `{PROJECT_ID}.{dataset_name}.{table_name}`
  """
  #print(sql)
  df_result = client.query(sql).to_dataframe()
  #display(df_result)
  return df_result['result'].iloc[0]

In [ ]:
def RunQuery(sql):
  import time

  #return True # return early for now

  job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
  query_job = client.query(sql, job_config=job_config)

  # Check on the progress by getting the job's updated state.
  query_job = client.get_job(
      query_job.job_id, location=query_job.location
  )
  print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

  while query_job.state != "DONE":
    time.sleep(5)
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
        )
    print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

  if query_job.error_result == None:
    return True
  else:
    return False

## Create Tables

In [ ]:
sql = f"""
CREATE TABLE IF NOT EXISTS `{PROJECT_ID}.{DATASET_ID}.company`
(
    company_id   INTEGER NOT NULL OPTIONS(description="Primary key."),
    company_name STRING NOT NULL OPTIONS(description="Name of the company."),
)
CLUSTER BY company_id;
"""

RunQuery(sql)

In [ ]:
sql = f"""
CREATE TABLE IF NOT EXISTS `{PROJECT_ID}.{DATASET_ID}.location`
(
    location_id INTEGER NOT NULL OPTIONS(description="Primary key."),
    company_id INTEGER NOT NULL OPTIONS(description="Foreign key: A valid value from the company table."),
    location_name STRING NOT NULL OPTIONS(description="Name of the location."),
    location_type STRING NOT NULL OPTIONS(description="Valid values for location_type are: 'Fixed', 'Mobile'"),
)
CLUSTER BY location_id;
"""

RunQuery(sql)

In [ ]:
sql = f"""
CREATE TABLE IF NOT EXISTS `{PROJECT_ID}.{DATASET_ID}.customer`
(
    customer_id INTEGER NOT NULL OPTIONS(description="Primary key."),
    company_id INTEGER NOT NULL OPTIONS(description="Foreign key: A valid value from the company table."),
    customer_name STRING NOT NULL OPTIONS(description="Name of the customer."),
    customer_yob INT NOT NULL OPTIONS(description="Customer year of birth"),
    customer_email STRING NOT NULL OPTIONS(description="Customer's email address"),
    customer_inception_date DATE NOT NULL OPTIONS(description="Date of first customer interaction")

)
CLUSTER BY customer_id;
"""

RunQuery(sql)

In [ ]:
sql = f"""
CREATE TABLE IF NOT EXISTS `{PROJECT_ID}.{DATASET_ID}.customer_profile`
(
    customer_id INTEGER NOT NULL OPTIONS(description="Primary key. Foreign key: Customer table."),
    customer_llm_summary STRING NOT NULL OPTIONS(description="LLM generated summary of customer data."),
    customer_lifetime_value STRING NOT NULL OPTIONS(description="Total sales for this customer."),
    customer_cluster_id INT NOT NULL OPTIONS(description="Clustering algorithm id."),
    customer_review_llm_summary STRING NOT NULL OPTIONS(description="LLM summary are all of the customer reviews."),
    customer_survey_llm_summary STRING NOT NULL OPTIONS(description="LLM summary are all of the customer surveys.")

)
CLUSTER BY customer_id;
"""

RunQuery(sql)

In [ ]:
sql = f"""
CREATE TABLE IF NOT EXISTS `{PROJECT_ID}.{DATASET_ID}.customer_survey`
(
    customer_survey_id INTEGER NOT NULL OPTIONS(description="Primary key."),
    customer_id INTEGER NOT NULL OPTIONS(description="Foreign key: Customer table"),
    survey_date DATE NOT NULL OPTIONS(description="The date of the survey"),
    question_1 STRING NOT NULL OPTIONS(description="A survey question."),
    response_1 STRING NOT NULL OPTIONS(description="The customers response."),
    question_2 STRING NOT NULL OPTIONS(description="A survey question."),
    response_2 STRING NOT NULL OPTIONS(description="The customers response."),
    question_3 STRING NOT NULL OPTIONS(description="A survey question."),
    response_3 STRING NOT NULL OPTIONS(description="The customers response.")
)
CLUSTER BY customer_id;
"""

RunQuery(sql)

In [ ]:
sql = f"""
CREATE TABLE IF NOT EXISTS `{PROJECT_ID}.{DATASET_ID}.customer_review`
(
    customer_review_id INTEGER NOT NULL OPTIONS(description="Primary key."),
    customer_id INTEGER NOT NULL OPTIONS(description="Foreign key: Customer table"),
    location_id INTEGER NOT NULL OPTIONS(description="Foreign key: Location table"),
    review_datetime TIMESTAMP NOT NULL OPTIONS(description="Date and time of the review."),
    review_text STRING NOT NULL OPTIONS(description="The customer's review of the coffee."),
    review_mp3 STRING OPTIONS(description="The GCS location of an attached MP3 file."),
    review_image STRING OPTIONS(description="The GCS location of an attached image file."),
    review_sentiment STRING OPTIONS(description="The sentiment of the review text."),
    social_media_source STRING NOT NULL OPTIONS(description="The social media site the review was posted on."),
    social_media_handle STRING NOT NULL OPTIONS(description="The customer's social media handle"),
    gen_ai_recommended_action STRING OPTIONS(description="Valid values for gen_ai_recommended_action are: 'Send Survey', 'Send Coupon'"),
    gen_ai_reponse STRING OPTIONS(description="The Generated response from the LLM."),
    human_response STRING OPTIONS(description="The human manually entered response."),
    response_sent_action STRING OPTIONS(description="Valid values for response_sent_action are: 'Sent LLM Response', 'Human called'"),
    response_sent_date TIMESTAMP OPTIONS(description="Date and time the response was sent.")

)
CLUSTER BY customer_id;
"""

RunQuery(sql)

In [ ]:
sql = f"""ALTER TABLE `{PROJECT_ID}.{DATASET_ID}.customer_review`
  ADD COLUMN IF NOT EXISTS llm_detected_theme JSON OPTIONS(description="The LLM detected themes in the customer review.");"""

RunQuery(sql)

In [ ]:
sql = f"""
CREATE TABLE IF NOT EXISTS `{PROJECT_ID}.{DATASET_ID}.menu`
(
    menu_id INTEGER NOT NULL OPTIONS(description="Primary key."),
    company_id INTEGER NOT NULL OPTIONS(description="Foreign key: Company table."),
    item_name STRING NOT NULL OPTIONS(description="The name of the coffee drink"),
    item_price STRING NOT NULL OPTIONS(description="The price of the coffee"),
    item_description STRING NOT NULL OPTIONS(description="The detailed description of the coffee drink"),
    item_size STRING NOT NULL OPTIONS(description="Valid Values: Small, Medium, Large")

)
CLUSTER BY menu_id;
"""

RunQuery(sql)

In [ ]:
sql = f"""
CREATE TABLE IF NOT EXISTS `{PROJECT_ID}.{DATASET_ID}.order`
(
    order_id INTEGER NOT NULL OPTIONS(description="Primary key."),
    location_id INTEGER NOT NULL OPTIONS(description="Foreign key: Location table."),
    customer_id INTEGER NOT NULL OPTIONS(description="Foreign key: Customer table."),
    order_datetime TIMESTAMP NOT NULL OPTIONS(description="The datetime the order was started."),
    order_completion_datetime TIMESTAMP NOT NULL OPTIONS(description="The datetime the order was completed.")

)
CLUSTER BY order_id;
"""

client.query(sql)

In [ ]:
sql = f"""
CREATE TABLE IF NOT EXISTS `{PROJECT_ID}.{DATASET_ID}.order_item`
(
    order_item_id INTEGER NOT NULL OPTIONS(description="Primary key."),
    order_id INTEGER NOT NULL OPTIONS(description="Foreign key: Order table"),
    menu_id INTEGER NOT NULL OPTIONS(description="Foreign key: Menu table"),
    quantity INTEGER NOT NULL OPTIONS(description="Number of items ordered")

)
CLUSTER BY order_id;
"""

RunQuery(sql)

In [ ]:
sql = f"""
CREATE TABLE IF NOT EXISTS `{PROJECT_ID}.{DATASET_ID}.coffee_farm`
(
    coffee_farm_id INTEGER NOT NULL OPTIONS(description="Primary key."),
    name STRING NOT NULL OPTIONS(description="The name of the coffee farm."),
    latitude FLOAT64 NOT NULL OPTIONS(description="The latitude of the coffee farm."),
    longitude FLOAT64 NOT NULL OPTIONS(description="The longitude of the coffee farm."),
    lat_long GEOGRAPHY NOT NULL OPTIONS(description="The latitude and longitude of the coffee farm."),
    contact_name STRING NOT NULL OPTIONS(description="The main contact person of the coffee farm."),
    contact_email STRING NOT NULL OPTIONS(description="The email address of the coffee farm."),
    contact_code STRING NOT NULL OPTIONS(description="Alpha 3 Code")
)
CLUSTER BY coffee_farm_id;
"""

RunQuery(sql)

In [ ]:
sql = f"""
CREATE TABLE IF NOT EXISTS `{PROJECT_ID}.{DATASET_ID}.coffee_processor`
(
    coffee_processor_id INTEGER NOT NULL OPTIONS(description="Primary key."),
    name STRING NOT NULL OPTIONS(description="The name of the coffee processing facility."),
    latitude FLOAT64 NOT NULL OPTIONS(description="The latitude of the coffee processing facility."),
    longitude FLOAT64 NOT NULL OPTIONS(description="The longitude of the coffee processing facility."),
    lat_long GEOGRAPHY NOT NULL OPTIONS(description="The latitude and longitude of the coffee processing facility."),
    contact_name STRING NOT NULL OPTIONS(description="The main contact person of the coffee processing facility."),
    contact_email STRING NOT NULL OPTIONS(description="The email address of the coffee processing facility."),
    contact_code STRING NOT NULL OPTIONS(description="Alpha 3 Code")
)
CLUSTER BY coffee_processor_id;
"""

RunQuery(sql)

In [ ]:
sql = f"""
CREATE TABLE IF NOT EXISTS `{PROJECT_ID}.{DATASET_ID}.coffee_roaster`
(
    coffee_roaster_id INTEGER NOT NULL OPTIONS(description="Primary key."),
    name STRING NOT NULL OPTIONS(description="The name of the coffee roaster facility."),
    latitude FLOAT64 NOT NULL OPTIONS(description="The latitude of the coffee roaster facility."),
    longitude FLOAT64 NOT NULL OPTIONS(description="The longitude of the coffee roaster facility."),
    lat_long GEOGRAPHY NOT NULL OPTIONS(description="The latitude and longitude of the coffee roaster facility."),
    contact_name STRING NOT NULL OPTIONS(description="The main contact person of the coffee roaster facility."),
    contact_email STRING NOT NULL OPTIONS(description="The email address of the coffee roaster facility."),
    contact_code STRING NOT NULL OPTIONS(description="Alpha 3 Code")
)
CLUSTER BY coffee_roaster_id;
"""

RunQuery(sql)

In [ ]:
sql = f"""
CREATE TABLE IF NOT EXISTS `{PROJECT_ID}.{DATASET_ID}.city`
(
    city_id INTEGER NOT NULL OPTIONS(description="Primary key."),
    city_name STRING NOT NULL OPTIONS(description="The name of the city."),
    country_code STRING NOT NULL OPTIONS(description="Alpha 3 Code."),
    latitude FLOAT64 NOT NULL OPTIONS(description="The latitude of the city."),
    longitude FLOAT64 NOT NULL OPTIONS(description="The longitude of the city."),
    lat_long GEOGRAPHY NOT NULL OPTIONS(description="The latitude and longitude of the city.")
)
CLUSTER BY city_id;
"""

RunQuery(sql)

sql = f"""
INSERT INTO `{PROJECT_ID}.{DATASET_ID}.city` VALUES
(1, 'New York City', 'USA', 40.7128, 74.0060, ST_GeogPoint(74.0060, 40.7128)),
(2, 'London', 'GBR', 51.5072, 0.1276, ST_GeogPoint(0.1276, 51.5072)),
(3, 'Tokyo', 'JPN', 35.6764, 139.6500, ST_GeogPoint(139.6500, 35.6764)),
(4, 'San Francisco', 'USA', 37.7749, 122.4194, ST_GeogPoint(122.4194, 37.7749));
"""

RunQuery(sql)

In [ ]:
sql = f"""
CREATE TABLE IF NOT EXISTS `{PROJECT_ID}.{DATASET_ID}.sales_forecast`
(
    sales_forecast_id INTEGER NOT NULL OPTIONS(description="Primary key."),
    forecast_date DATE NOT NULL OPTIONS(description="The date for the forecasted sales"),
    sales_forecast_amount FLOAT64 NOT NULL OPTIONS(description="The projected sales for the city."),
    city_id INTEGER NOT NULL OPTIONS(description="Foreign key: City table.")
)
CLUSTER BY sales_forecast_id;
"""

RunQuery(sql)

In [ ]:
sql = f"""
CREATE TABLE IF NOT EXISTS `{PROJECT_ID}.{DATASET_ID}.marketing_campaign`
(
    marketing_id INTEGER NOT NULL OPTIONS(description="Primary key."),
    marketing_campaign_name STRING NOT NULL OPTIONS(description="The cost for the campaign."),
    marketing_cost FLOAT64 NOT NULL OPTIONS(description="The cost for the campaign."),
    number_of_new_customers INT64 NOT NULL OPTIONS(description="The projected sales for the city.")
)
CLUSTER BY marketing_id;
"""

RunQuery(sql)

sql = f"""
INSERT INTO `{PROJECT_ID}.{DATASET_ID}.marketing_campaign` VALUES
(1, 'Google Ads', 15000, 1500),
(2, 'Billboards', 4000, 200);
"""

RunQuery(sql)

## Company Table

Create the names for the coffee companies.

*   Create the names outside of the SQL statement.  This provides more control over the names.
*   Use the names for the SQL prompt.



In [ ]:
company_count = 10

table_name = "company"
primary_key = "company_id"

schema = GetTableSchema(DATASET_ID, table_name)

In [ ]:
prompt = f"""Generate {company_count} creative names and return in the below json format.
- The name should be new and not a name that is already used by an existing coffee company.
- The name should be related to coffee.
- The name should be related to a food truck type of service.

JSON format: [ "value" ]
Sample JSON Response: [ "value1", "value2" ]
"""

llm_valid_execution = False
while llm_valid_execution == False:
  try:
    company_names = LLM(prompt, True, max_output_tokens=1024, temperature=1, top_p=1, top_k=40)
    llm_valid_execution = True
  except Exception as error:
    print("An error occurred:", error)

print(f"company_names: {company_names}")

In [ ]:
# Override (hardcoded)
company_names = """[
  "BeanStreet Brews",
  "MochaWheels Cafe",
  "Brew 'n Go Bistro",
  "Rolling Roast Express",
  "JavaJolt Junction",
  "Café on Wheels",
  "Espresso Eats Mobile",
  "CuppaCruiser",
  "The Roaming Roastery",
  "StreetBean Delights"
]"""

company_names = company_names.replace("'","\\'") # Deal with single quotes in output

In [ ]:
company_names

In [ ]:
starting_value = GetStartingValue(DATASET_ID, table_name, primary_key)

prompt=f"""
You are a database engineer and need to generate data for a table for the below schema.
- The schema is for a Google Cloud BigQuery Table.
- The table name is "{PROJECT_ID}.{DATASET_ID}.{table_name}".
- Read the description of each field for valid values.
- Do not preface the response with any special characters or 'sql'.
- Generate {company_count} insert statements for this table.
- Valid values for company_name are: {company_names}
- The starting value of the field {primary_key} is {starting_value}.

Example 1: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Sample'),(2, 'Sample');
Example 2: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Data'),(2, 'Data'),(3, 'Data');

Schema: {schema}
"""

llm_valid_execution = False
while llm_valid_execution == False:
  try:
    sql = LLM(prompt, False, max_output_tokens=1024, temperature=1, top_p=1, top_k=40)
    print("---------------------------------")
    print("sql: ", sql)
    print("---------------------------------")
    llm_valid_execution = RunQuery(sql)
  except Exception as error:
    print("An error occurred:", error)

## Location Table

In [ ]:
location_count = 50
location_list = "New York, San Francisco, Miami"

table_name = "company"
field_name = "company_id"
company_ids = GetForeignKeys(DATASET_ID, table_name, field_name)

table_name = "location"
primary_key = "location_id"

schema = GetTableSchema(DATASET_ID, table_name)

In [ ]:
prompt = f"""Generate {location_count} creative names and return in the below json format.
- The name should be new and not a name that is already used by an existing coffee company.
- The name should be related to mobile coffee trucks that travel around the city.
- The name should be related the following locations: {location_list}

JSON format: [ "value" ]
Sample JSON Response: [ "value1", "value2" ]
"""

llm_valid_execution = False
while llm_valid_execution == False:
  try:
    location_names = LLM(prompt, True, max_output_tokens=1024, temperature=1, top_p=1, top_k=40)
    llm_valid_execution = True
  except Exception as error:
    print("An error occurred:", error)

In [ ]:
# Override (hardcoded)

location_names = """[
  "CitySips Roaming Cafe",
  "JavaJourney Express",
  "UrbanCaffeine Cruiser",
  "CafeWheels NY",
  "Golden Gate Grind Mobile",
  "Sunshine Sips on Wheels",
  "Big Apple Brew Bus",
  "Bay Brew Hauler",
  "Magic City Mocha Mobile",
  "Metropolis Mug Mover",
  "New York Nectar Nomad",
  "SFO Street Sips",
  "MiaMornings Mobile",
  "CityBeans Roam-uccino",
  "Sunrise City Sipper",
  "Gotham Grind on Wheels",
  "Bay Area Bean Bus",
  "Mia Mochaccino Mobile",
  "Cityscape Sip Stop",
  "Transit Brew Buggy",
  "Fog City Fueler",
  "Miami Metro Mugs",
  "NY Espresso Express",
  "Golden Grind Getter",
  "Sunny Side Sips Shuttle",
  "Empire Espresso Explorer",
  "SF Sidewalk Sipper",
  "Beachside Brew Bounder",
  "Urban Sipper's Shuttle",
  "NYC Nomadic Nectar",
  "Golden Bridge Brewmobile",
  "Sunny State Sipster",
  "Cafe Cruiser Central",
  "NY Neighborhood Nectar",
  "Frisco Fuel on Wheels",
  "MiaMug Mobility",
  "Metropolitan Mochaccino",
  "CitySips Street Surfer",
  "Golden Gate Gourmet Glide",
  "Beach Breeze Brew Bus",
  "City Roast Cruiser",
  "NYC Urban Uplifter",
  "Frisco Fresh Brews",
  "Miami Magic Mugs",
  "Coffee Cart Connection",
  "Empire City Espresso Explorer",
  "Golden Glow Grind Rover",
  "Sun-Kissed Sip & Go",
  "CityLife Latte Lorry",
  "Cityscape Sipper Shuttle"
]"""

location_names = location_names.replace("'","\\'") # Deal with single quotes in output

In [ ]:
starting_value = GetStartingValue(DATASET_ID, table_name, primary_key)

prompt=f"""
You are a database engineer and need to generate data for a table for the below schema.
- The schema is for a Google Cloud BigQuery Table.
- The table name is "{PROJECT_ID}.{DATASET_ID}.{table_name}".
- Read the description of each field for valid values.
- Do not preface the response with any special characters or 'sql'.
- Generate {location_count} insert statements for this table.
- Valid values for location_name are: {location_names}
- Valid values for company_id are: {company_ids}
- The starting value of the field {primary_key} is {starting_value}.

Example 1: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Sample'),(2, 'Sample');
Example 2: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Data'),(2, 'Data'),(3, 'Data');

Schema: {schema}
"""

llm_valid_execution = False
while llm_valid_execution == False:
  try:
    sql = LLM(prompt, False, max_output_tokens=1024, temperature=1, top_p=1, top_k=40)
    print("---------------------------------")
    print("sql: ", sql)
    print("---------------------------------")
    llm_valid_execution = RunQuery(sql)
  except Exception as error:
    print("An error occurred:", error)

## Customer Table

In [ ]:
customer_count = 20
country = "United States"

table_name = "company"
field_name = "company_id"
company_ids = GetForeignKeys(DATASET_ID, table_name, field_name)

table_name = "customer"
primary_key = "customer_id"

schema = GetTableSchema(DATASET_ID, table_name)

In [ ]:
loop_count = 10
loop_index = 1

while loop_index <= loop_count:
  print(f"loop_index: {loop_index} | loop_count: {loop_count}")
  starting_value = GetStartingValue(DATASET_ID, table_name, primary_key)

  prompt=f"""
  You are a database engineer and need to generate data for a table for the below schema.
  - The schema is for a Google Cloud BigQuery Table.
  - The table name is "{PROJECT_ID}.{DATASET_ID}.{table_name}".
  - Read the description of each field for valid values.
  - Do not preface the response with any special characters or 'sql'.
  - Generate {customer_count} insert statements for this table.
  - The customer_inception_date is a date and should be within the past 5 years.
  - The customer_name should be names used in the country {country} and be a first and last name.
  - Valid values for company_id are: {company_ids}
  - The starting value of the field {primary_key} is {starting_value}.

  Example 1: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Sample'),(2, 'Sample');
  Example 2: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Data'),(2, 'Data'),(3, 'Data');

  Schema: {schema}
  """


  llm_valid_execution = False
  while llm_valid_execution == False:
    try:
      sql = LLM(prompt, False, max_output_tokens=1024, temperature=1, top_p=1, top_k=40)
      print("---------------------------------")
      print("sql: ", sql)
      print("---------------------------------")
      llm_valid_execution = RunQuery(sql)
      loop_index = loop_index + 1
    except Exception as error:
      print("An error occurred:", error)

## Customer Survey Table

* Generate a list of survey coffee questions
* Save these, so we re-use in each loop
* Generate survey responses



In [ ]:
customer_survey_count = 3

prompt = f"""You work in the marketing department and are gathering feedback on coffee served to customers.
Generate {customer_survey_count} random coffee survey questions and return in the below JSON format.
- Do not include any special characters in the json.
- Do not include ```json in the output.

JSON format: [ "value" ]
Sample JSON Response: [ "value1", "value2" ]
"""

llm_valid_execution = False
while llm_valid_execution == False:
  try:
    generatcustomer_surveysed_customer_surveys = LLM(prompt, True, max_output_tokens=1024, temperature=1, top_p=1, top_k=40)
    llm_valid_execution = True
  except Exception as error:
    print("An error occurred:", error)

# Hard coded so we ask the same questions (for now)
customer_surveys = '["How satisfied were you with the taste of our coffee?", "How likely are you to recommend our coffee to a friend?", "Which of the following best describes your overall experience with our coffee?"]'

# Load the json do we index
customer_surveys_json = json.loads(customer_surveys)

In [ ]:
rows_of_data_to_generate = 5

table_name = "customer"
field_name = "customer_id"
customer_ids = GetForeignKeys(DATASET_ID, table_name, field_name)

table_name = "customer_survey"
primary_key = "customer_survey_id"

schema = GetTableSchema(DATASET_ID, table_name)

In [ ]:
loop_count = 10
loop_index = 1

while loop_index <= loop_count:
  print(f"loop_index: {loop_index} | loop_count: {loop_count}")
  starting_value = GetStartingValue(DATASET_ID, table_name, primary_key)

  prompt=f"""
  You are a database engineer and need to generate data for a table for the below schema.
  - The schema is for a Google Cloud BigQuery Table.
  - The table name is "{PROJECT_ID}.{DATASET_ID}.{table_name}".
  - Read the description of each field for valid values.
  - Do not preface the response with any special characters or 'sql'.
  - Generate {rows_of_data_to_generate} insert statements for this table.
  - Valid values for customer_id are: {customer_ids}
  - The survey_date is a date and should be within the past 5 years.
  - Valid values for question_1 is {customer_surveys_json[0]}
  - Valid values for question_2 is {customer_surveys_json[1]}
  - Valid values for question_3 is {customer_surveys_json[2]}
  - The response for each question should be 2 to 20 words.
  - The response can have a postive, neutral or negative sentiment.
  - The starting value of the field {primary_key} is {starting_value}.
  - Escape single quotes with a backslash.  Example: Adam's Answer: Adam\'s

  Example 1: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Sample'),(2, 'Sample');
  Example 2: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Data'),(2, 'Data'),(3, 'Data');

  Schema: {schema}
  """

  llm_valid_execution = False
  while llm_valid_execution == False:
    try:
      sql = LLM(prompt, False, max_output_tokens=1024, temperature=1, top_p=1, top_k=40)
      print("---------------------------------")
      print("sql: ", sql)
      print("---------------------------------")
      llm_valid_execution = RunQuery(sql)
      loop_index = loop_index + 1
    except Exception as error:
      print("An error occurred:", error)

## Customer Review Table

### Create customer reviews

In [ ]:
rows_of_data_to_generate = 3

table_name = "customer"
field_name = "customer_id"
customer_ids = GetForeignKeys(DATASET_ID, table_name, field_name)

table_name = "location"
field_name = "location_id"
location_ids = GetForeignKeys(DATASET_ID, table_name, field_name)

table_name = "customer_review"
primary_key = "customer_review_id"

schema = GetTableSchema(DATASET_ID, table_name)

In [ ]:
# location_ids='1,11,21'

In [ ]:
import random
loop_count = 1000
loop_index = 1

while loop_index <= loop_count:
  print(f"loop_index: {loop_index} | loop_count: {loop_count}")
  starting_value = GetStartingValue(DATASET_ID, table_name, primary_key)

  if random.random() < .25:
    prompt=f"""
    You are a database engineer and need to generate data for a table for the below schema.
    You need to generate reviews for customers who have purchased your brewed coffee.
    Write a negative in first person based upon the following: "Bad Service","Long Wait Time","Slow Service","Dirty","Overpriced","Overcrowded","Noisy Location","Lack of Allergan Information","Inconsistent Quality","Lack of Seating","No Flavor","Too weak","Too strong","Too bitter","Limited Menu"
    - The schema is for a Google Cloud BigQuery Table.
    - The table name is "{PROJECT_ID}.{DATASET_ID}.{table_name}".
    - Read the description of each field for valid values.
    - Do not preface the response with any special characters or 'sql'.
    - Generate {rows_of_data_to_generate} insert statements for this table.
    - Valid values for customer_id are: {customer_ids}
    - Valid values for location_id are: {location_ids}
    - The review_datetime is a date and should be within the past 5 years.
    - The response for each question should be 20 to 100 words.
    - The starting value of the field {primary_key} is {starting_value}.
    - Escape single quotes with a backslash.  Example: Adam's Answer: Adam\'s
    - Only generate data for these fields: customer_review_id, customer_id, location_id, review_datetime, review_text, social_media_source, social_media_handle

    Examples:
    Example 1: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Sample'),(2, 'Sample');
    Example 2: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Data'),(2, 'Data'),(3, 'Data');

    Schema: {schema}
    """
  else:
    prompt=f"""
    You are a database engineer and need to generate data for a table for the below schema.
    You need to generate reviews for customers who have purchased your brewed coffee.
    Write a positive or neutral review in first person based upon the following: "Good Service","Short Wait Time","Fast Service","Clean","Good value","Cozy Seating Areas","Quite Location","Variety of Milk Alternatives","Consistent Quality","Lots of places to sit","Lots of Flavor","Good Taste","Good Selection"
    - The schema is for a Google Cloud BigQuery Table.
    - The table name is "{PROJECT_ID}.{DATASET_ID}.{table_name}".
    - Read the description of each field for valid values.
    - Do not preface the response with any special characters or 'sql'.
    - Generate {rows_of_data_to_generate} insert statements for this table.
    - Valid values for customer_id are: {customer_ids}
    - The review_datetime is a date and should be within the past 5 years.
    - The response for each question should be 20 to 100 words.
    - The starting value of the field {primary_key} is {starting_value}.
    - Escape single quotes with a backslash.  Example: Adam's Answer: Adam\'s
    - Only generate data for these fields: customer_review_id, customer_id, location_id, review_datetime, review_text, social_media_source, social_media_handle

    Examples:
    Example 1: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Sample'),(2, 'Sample');
    Example 2: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Data'),(2, 'Data'),(3, 'Data');

    Schema: {schema}
    """

  llm_valid_execution = False
  while llm_valid_execution == False:
    try:
      sql = LLM(prompt, False, max_output_tokens=1024, temperature=1, top_p=1, top_k=40)
      print("---------------------------------")
      print("sql: ", sql)
      print("---------------------------------")
      llm_valid_execution = RunQuery(sql)
      loop_index = loop_index + 1
    except Exception as error:
      print("An error occurred:", error)

### Score the Sentiment

In [ ]:
sql = """SELECT customer_review_id,
                review_text
          FROM `data-analytics-preview.coffee_curated.customer_review`
         WHERE review_sentiment IS NULL
        ORDER BY customer_review_id
"""

# Fields to update
# review_sentiment,
# gen_ai_recommended_action,
# gen_ai_reponse,
# human_response,
# response_sent_action,
# response_sent_date

df_process = client.query(sql).to_dataframe()

for row in df_process.itertuples():
  customer_review_id = row.customer_review_id
  review_text = row.review_text

  llm_valid_execution = False
  while llm_valid_execution == False:
    try:
      prompt=f"""
      For the given review classify the sentiment as Positive, Neutral or Negative.
      Review: {review_text}
      """
      review_sentiment = LLM(prompt, False, max_output_tokens=10, temperature=0, top_p=0, top_k=1)

      sql = f"""UPDATE `data-analytics-preview.coffee_curated.customer_review`
                  SET review_sentiment = '{review_sentiment}'
                WHERE customer_review_id = {customer_review_id}
      """

      print (sql)

      llm_valid_execution = RunQuery(sql)
      llm_valid_execution = True
    except Exception as error:
      print("An error occurred:", error)

### Gen AI Response

In [ ]:
sql = """SELECT customer_review_id,
                review_text
          FROM `data-analytics-preview.coffee_curated.customer_review`
         WHERE gen_ai_reponse IS NULL
        ORDER BY customer_review_id"""

df_process = client.query(sql).to_dataframe()

for row in df_process.itertuples():
  customer_review_id = row.customer_review_id
  review_text = row.review_text

  llm_valid_execution = False
  while llm_valid_execution == False:
    try:
      prompt=f"""
      Generate responses to the below customer review who purchased coffee and return the results the below json format.
      The review can be positive, negative, or neutral.
      Provide a variety of responses, including thanking customers for positive reviews, addressing concerns in negative reviews, and engaging with neutral reviews.
      Please generate at least 5 different responses.

      JSON format: [ "value" ]
      Sample JSON Response: [ "response 1", "response 2", "response 3", "response 4", "response 5" ]

      Review: {review_text}"""

      json_result = LLM(prompt, True, max_output_tokens=1024, temperature=0, top_p=0, top_k=1)
      print(f"json_result: {json_result}")

      if json_result == None:
        llm_valid_execution = False
      else:
        sql = f"""UPDATE `data-analytics-preview.coffee_curated.customer_review`
                    SET gen_ai_reponse = '{json_result}'
                  WHERE customer_review_id = {customer_review_id}
        """

        print(f"sql: {sql}")

        llm_valid_execution = RunQuery(sql)
    except Exception as error:
      print("An error occurred:", error)

### Gen AI Recommended Action

In [ ]:
sql = """SELECT customer_review_id,
                review_text
          FROM `data-analytics-preview.coffee_curated.customer_review`
         WHERE gen_ai_recommended_action IS NULL
        ORDER BY customer_review_id"""

df_process = client.query(sql).to_dataframe()

for row in df_process.itertuples():
  customer_review_id = row.customer_review_id
  review_text = row.review_text

  llm_valid_execution = False
  while llm_valid_execution == False:
    try:
      prompt="""
      Select one of the following actions based upon the below customer review who purchased coffee.
      - First randomly sort the actions.
      - Select the best action based upon the sentiment of the review.
      - It is okay to use the action "Send the customer a coupon" for both positive and negative reviews.
      - Return the results the below json format.
      - Do not include any special characters or "```json" in the json output

      Actions
      - "Thank the Customer"
      - "Apologize to the Customer"
      - "Send the customer a coupon"
      - "Call the customer"
      - "Promote Additional Products"
      - "Promise to Investigate"
      - "Encourage More Reviews"
      - "Invite Further Engagement"
      - "Reshare the review on other social media"

      JSON format: { "action" : "value", "explaination" : "llm explaination" }
      Sample JSON Response: { "action" : "Call the customer", "explaination" : "The customer left their phone number in the review." }
      Sample JSON Response: { "action" : "Encourage More Reviews", "explaination" : "Thanks for the review, please keep posting." }

      Review:"""
      prompt = prompt + review_text

      json_result = LLM(prompt, True, max_output_tokens=1024, temperature=0, top_p=0, top_k=1)
      print(f"json_result: {json_result}")

      if json_result == None:
        llm_valid_execution = False
      else:
        sql = f"""UPDATE `data-analytics-preview.coffee_curated.customer_review`
                    SET gen_ai_recommended_action = '{json_result}'
                  WHERE customer_review_id = {customer_review_id}
        """

        print(f"sql: {sql}")

        llm_valid_execution = RunQuery(sql)
    except Exception as error:
      print("An error occurred:", error)

### Detect Customer Themes

In [ ]:
sql = """SELECT customer_review_id,
                review_text
          FROM `data-analytics-preview.coffee_curated.customer_review`
         WHERE llm_detected_theme IS NULL
        ORDER BY customer_review_id"""

df_process = client.query(sql).to_dataframe()

for row in df_process.itertuples():
  customer_review_id = row.customer_review_id
  review_text = row.review_text

  llm_valid_execution = False
  while llm_valid_execution == False:
    try:
      prompt="""
      Classify the below customer review as one or more of the below themes.
      - Return the results the below json format.
      - Include an explaination for selecting each theme.
      - Do not include double quotes in the explaination.
      - Do not include any special characters, double quotes or "```json" in the json output.

      Themes
      - "Bad Service"
      - "Long Wait Time"
      - "Slow Service"
      - "Dirty"
      - "Overpriced"
      - "Overcrowded"
      - "Noisy Location"
      - "Lack of Allergan Information"
      - "Inconsistent Quality"
      - "Lack of Seating"
      - "No Flavor"
      - "Too weak"
      - "Too strong"
      - "Too bitter"
      - "Limited Menu"
      - "Good Service"
      - "Short Wait Time"
      - "Fast Service"
      - "Clean"
      - "Good value"
      - "Cozy Seating Areas"
      - "Quite Location"
      - "Variety of Milk Alternatives"
      - "Consistent Quality"
      - "Lots of places to sit"
      - "Lots of Flavor"
      - "Good Taste"
      - "Good Selection"

      JSON format: [{ "theme" : "value", "explaination" : "llm explaination" }]
      Sample JSON Response: [{ "theme" : "Fast Service", "explaination" : "The customer got their order fast." }]
      Sample JSON Response: [{ "theme" : "Overpriced", "explaination" : "The customer said it was too expensive." }]

      Review:"""
      prompt = prompt + review_text

      json_result = LLM(prompt, True, max_output_tokens=1024, temperature=0, top_p=0, top_k=1)
      print(f"json_result: {json_result}")

      if json_result == None:
        llm_valid_execution = False
      else:
        sql = f"""UPDATE `data-analytics-preview.coffee_curated.customer_review`
                    SET llm_detected_theme = JSON'{json_result}'
                  WHERE customer_review_id = {customer_review_id}
        """

        print(f"sql: {sql}")

        llm_valid_execution = RunQuery(sql)
    except Exception as error:
      print("An error occurred:", error)

## Menu Table

In [ ]:
table_name = "company"
field_name = "company_id"
company_ids = GetForeignKeys(DATASET_ID, table_name, field_name)

table_name = "menu"
primary_key = "menu_id"

schema = GetTableSchema(DATASET_ID, table_name)

In [ ]:
loop_count = 10
loop_index = 1
menu_count = 10

while loop_index <= loop_count:
  print(f"loop_index: {loop_index} | loop_count: {loop_count}")

  # Get Menu Names
  menu_count = 5 # We mulitple this by 3 to get 15 (one for small, medium and large sizes)

  table_name = "menu"
  field_name = "item_name"
  existing_values = GetDistinctValues(DATASET_ID, table_name, field_name)

  prompt = f"""Generate {menu_count} different coffee drink names and return in the below json format.
  - The name can be an existing coffee drink or something new.
  - The name should be related to coffee.
  - Do not use any of these names: [{existing_values}]
  - Do not number the results.

  JSON format: [ "value" ]
  Sample JSON Response: [ "value1", "value2" ]
  """

  llm_valid_execution = False
  while llm_valid_execution == False:
    try:
      menu_names = LLM(prompt, True, max_output_tokens=1024, temperature=1, top_p=1, top_k=40)
      llm_valid_execution = True
    except Exception as error:
      print("An error occurred:", error)


  # Insert data
  starting_value = GetStartingValue(DATASET_ID, table_name, primary_key)

  prompt=f"""
  You are a database engineer and need to generate data for a table for the below schema.
  - The schema is for a Google Cloud BigQuery Table.
  - The table name is "{PROJECT_ID}.{DATASET_ID}.{table_name}".
  - Read the description of each field for valid values.
  - Do not preface the response with any special characters or 'sql'.
  - Generate {menu_count * 3} total rows for this table.
  - Valid values for company_id are: {company_ids}
  - Valid values for item_name are: {menu_names}
  - The starting value of the field {primary_key} is {starting_value}.
  - Create a Small, Medium and Large size for each item_name.  The same company_id should be used as well for all 3 sizes.

  Example 1: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Sample'),(2, 'Sample');
  Example 2: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Data'),(2, 'Data'),(3, 'Data');

  Schema: {schema}
  """

  llm_valid_execution = False
  while llm_valid_execution == False:
    try:
      sql = LLM(prompt, False, max_output_tokens=1024, temperature=1, top_p=1, top_k=40)
      print("---------------------------------")
      print("sql: ", sql)
      print("---------------------------------")
      llm_valid_execution = RunQuery(sql)
      loop_index = loop_index + 1
    except Exception as error:
      print("An error occurred:", error)

In [ ]:
loop_count = 1
loop_index = 1
menu_count = 20

while loop_index <= loop_count:
  print(f"loop_index: {loop_index} | loop_count: {loop_count}")

  table_name = "menu"
  field_name = "item_name"
  existing_values = GetDistinctValues(DATASET_ID, table_name, field_name)

  prompt = f"""Generate {menu_count} different foods that you would buy with coffee and return in the below json format.
  - The name can be an existing coffee drink or something new.
  - The name should be related to coffee.
  - Do not use any of these names: [{existing_values}]
  - Do not number the results.

  JSON format: [ "value" ]
  Sample JSON Response: [ "value1", "value2" ]
  """

  llm_valid_execution = False
  while llm_valid_execution == False:
    try:
      #menu_names = LLM(prompt, True, max_output_tokens=1024, temperature=1, top_p=1, top_k=40)
      llm_valid_execution = True
    except Exception as error:
      print("An error occurred:", error)

  menu_names = """
  ["Cinnamon Roll",
  "Espresso Brownie",
  "Coffee Tiramisu",
  "Cafe Latte Donut",
  "Coffee Glazed Bacon",
  "Hazelnut Croissant",
  "Mocha Pancakes",
  "Coffee-infused Oatmeal",
  "Cafe Muffin",
  "Espresso Chocolate Truffle",
  "Caffeine Chip Cookies",
  "Mocha Almond Croissant",
  "Coffee Bean Scones",
  "Java Chip Pancakes",
  "Hazelnut Latte Donut",
  "Cappuccino Cheesecake",
  "Coffee Tiramisu",
  "Caramel Macchiato Muffin",
  "Coffee-Infused Chocolate Truffles",
  "Coffee Bean Ice Cream"]"""
  # Insert data
  starting_value = GetStartingValue(DATASET_ID, table_name, primary_key)

  prompt=f"""
  You are a database engineer and need to generate data for a table for the below schema.
  - The schema is for a Google Cloud BigQuery Table.
  - The table name is "{PROJECT_ID}.{DATASET_ID}.{table_name}".
  - Read the description of each field for valid values.
  - Do not preface the response with any special characters or 'sql'.
  - Generate {menu_count} total rows for this table.
  - Valid values for company_id are: {company_ids}
  - Valid values for item_name are: {menu_names}
  - The starting value of the field {primary_key} is {starting_value}.
  - Create a Small, Medium and Large size for each item_name.  The same company_id should be used as well for all 3 sizes.

  Example 1: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Sample'),(2, 'Sample');
  Example 2: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Data'),(2, 'Data'),(3, 'Data');

  Schema: {schema}
  """

  llm_valid_execution = False
  while llm_valid_execution == False:
    try:
      sql = LLM(prompt, False, max_output_tokens=1024, temperature=1, top_p=1, top_k=40)
      print("---------------------------------")
      print("sql: ", sql)
      print("---------------------------------")
      llm_valid_execution = RunQuery(sql)
      loop_index = loop_index + 1
    except Exception as error:
      print("An error occurred:", error)

## Order Table

In [ ]:
order_count = 10

table_name = "customer"
field_name = "customer_id"
max_customer_id = GetMaximumValue(DATASET_ID, table_name, field_name)

table_name = "location"
field_name = "location_id"
max_location_id = GetMaximumValue(DATASET_ID, table_name, field_name)

table_name = "order"
primary_key = "order_id"

schema = GetTableSchema(DATASET_ID, table_name)

In [ ]:
loop_count = 50
loop_index = 1

while loop_index <= loop_count:
  print(f"loop_index: {loop_index} | loop_count: {loop_count}")
  starting_value = GetStartingValue(DATASET_ID, table_name, primary_key)

  prompt=f"""
  You are a database engineer and need to generate data for a table for the below schema.
  - The schema is for a Google Cloud BigQuery Table.
  - The table name is "{PROJECT_ID}.{DATASET_ID}.{table_name}".
  - Read the description of each field for valid values.
  - Do not preface the response with any special characters or 'sql'.
  - Generate {order_count} insert statements for this table.
  - The order_datetime is a date and should be within the past 1 year.
  - The order_completion_datetime should be within 60 to 900 seconds of the order_datetime.
  - Valid values for location_id between 1 and {max_location_id}.
  - Valid values for customer_id between 1 and {max_customer_id}.
  - The starting value of the field {primary_key} is {starting_value}.
  - Timestamps should use this format: 2020-06-02 23:57:12.120174 UTC.

  Example 1: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Sample'),(2, 'Sample');
  Example 2: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Data'),(2, 'Data'),(3, 'Data');

  Schema: {schema}
  """

  llm_valid_execution = False
  while llm_valid_execution == False:
    try:
      sql = LLM(prompt, False, max_output_tokens=1024, temperature=1, top_p=1, top_k=40)
      sql = sql.replace("\\'","'")
      print("---------------------------------")
      print("sql: ", sql)
      print("---------------------------------")
      llm_valid_execution = RunQuery(sql)
      loop_index = loop_index + 1
    except Exception as error:
      print("An error occurred:", error)


## Order Item Table

In [ ]:
order_item_count = 3

In [ ]:
table_name = "menu"
field_name = "menu_id"
max_menu_id = GetMaximumValue(DATASET_ID, table_name, field_name)

table_name = "order"
field_name = "order_id"
max_order_id = GetMaximumValue(DATASET_ID, table_name, field_name)

table_name = "order_item"
primary_key = "order_item_id"

schema = GetTableSchema(DATASET_ID, table_name)

In [ ]:
loop_count = 100
loop_index = 1

while loop_index <= loop_count:
  print(f"loop_index: {loop_index} | loop_count: {loop_count}")
  starting_value = GetStartingValue(DATASET_ID, table_name, primary_key)

  max_order_item_id = GetMaximumValue(DATASET_ID, table_name, "order_id")

  if max_order_item_id > max_order_id:
    print("Breaking out of loop since we have items for each order.")
    break

  order_id_to_generate_data = max_order_item_id + 1
  print(f"order_id_to_generate_data: {order_id_to_generate_data}")

  prompt=f"""
  You are a database engineer and need to generate data for a table for the below schema.
  - The schema is for a Google Cloud BigQuery Table.
  - The table name is "{PROJECT_ID}.{DATASET_ID}.{table_name}".
  - Read the description of each field for valid values.
  - Do not preface the response with any special characters or 'sql'.
  - Generate {order_item_count} insert statements for this table.
  - You can have 1 to 10 items for a single order_id.
  - The order_id should use the value of: {order_id_to_generate_data}.
  - Valid values for menu_id between 1 and {max_menu_id}.
  - Valid values for quantity as between 1 and 4.
  - The starting value of the field {primary_key} is {starting_value}.

  Example 1: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Sample'),(2, 'Sample');
  Example 2: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Data'),(2, 'Data'),(3, 'Data');

  Schema: {schema}
  """


  llm_valid_execution = False
  while llm_valid_execution == False:
    try:
      sql = LLM(prompt, False, max_output_tokens=1024, temperature=1, top_p=1, top_k=40)
      print("---------------------------------")
      print("sql: ", sql)
      print("---------------------------------")
      llm_valid_execution = RunQuery(sql)
      loop_index = loop_index + 1
    except Exception as error:
      print("An error occurred:", error)

## Coffee Farm Table

In [ ]:
coffee_farm_count = 10

table_name = "coffee_farm"
primary_key = "coffee_farm_id"

schema = GetTableSchema(DATASET_ID, table_name)

In [ ]:
table_name = "coffee_farm"
field_name = "name"
existing_values = GetDistinctValues(DATASET_ID, table_name, field_name)

prompt = f"""Generate {coffee_farm_count} creative names and return in the below json format.
- The name should be new and not a name that is already used by an existing coffee company.
- The name should be related to coffee farms that grow coffee beans.
- Do not use the following values: {existing_values}

JSON format: [ "value" ]
Sample JSON Response: [ "value1", "value2" ]
"""

llm_valid_execution = False
while llm_valid_execution == False:
  try:
    coffee_farm_names = LLM(prompt, True, max_output_tokens=1024, temperature=1, top_p=1, top_k=40)
    llm_valid_execution = True
  except Exception as error:
    print("An error occurred:", error)

print(f"coffee_farm_names: {coffee_farm_names}")

In [ ]:
starting_value = GetStartingValue(DATASET_ID, table_name, primary_key)

prompt=f"""
You are a database engineer and need to generate data for a table for the below schema.
- The schema is for a Google Cloud BigQuery Table.
- The table name is "{PROJECT_ID}.{DATASET_ID}.{table_name}".
- Read the description of each field for valid values.
- Do not preface the response with any special characters or 'sql'.
- Generate {coffee_farm_count} insert statements for this table.
- The starting value of the field {primary_key} is {starting_value}.
- Valid values for name are: {coffee_farm_names}
- Use the World Geodetic System (WGS) for the latitude and longitude values.
- When inserting GEOGRAPHY data types use the BigQuery function ST_GEOGPOINT.
- Escape single quotes with a backslash.  Example: Adam's Answer: Adam\'s

Example 1: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Sample'),(2, 'Sample');
Example 2: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Data'),(2, 'Data'),(3, 'Data');

Schema: {schema}
"""

llm_valid_execution = False
while llm_valid_execution == False:
  try:
    sql = LLM(prompt, False, max_output_tokens=1024, temperature=1, top_p=1, top_k=40)
    print("---------------------------------")
    print("sql: ", sql)
    print("---------------------------------")
    llm_valid_execution = RunQuery(sql)
  except Exception as error:
    print("An error occurred:", error)

## Coffee Processor Table

In [ ]:
coffee_processor_count = 10

table_name = "coffee_processor"
primary_key = "coffee_processor_id"

schema = GetTableSchema(DATASET_ID, table_name)

In [ ]:
table_name = "coffee_processor"
field_name = "name"
existing_values = GetDistinctValues(DATASET_ID, table_name, field_name)

prompt = f"""Generate {coffee_processor_count} creative names and return in the below json format.
- The name should be new and not a name that is already used by an existing coffee company.
- The name should be related to coffee processing facilities that process the raw beans.
- Do not use the following values: {existing_values}

JSON format: [ "value" ]
Sample JSON Response: [ "value1", "value2" ]
"""

llm_valid_execution = False
while llm_valid_execution == False:
  try:
    coffee_processor_names = LLM(prompt, True, max_output_tokens=1024, temperature=1, top_p=1, top_k=40)
    llm_valid_execution = True
  except Exception as error:
    print("An error occurred:", error)

print(f"coffee_processor_names: {coffee_processor_names}")

In [ ]:
starting_value = GetStartingValue(DATASET_ID, table_name, primary_key)

prompt=f"""
You are a database engineer and need to generate data for a table for the below schema.
- The schema is for a Google Cloud BigQuery Table.
- The table name is "{PROJECT_ID}.{DATASET_ID}.{table_name}".
- Read the description of each field for valid values.
- Do not preface the response with any special characters or 'sql'.
- Generate a single INSERT statement with {coffee_processor_count} rows of data.
- The starting value of the field {primary_key} is {starting_value}.
- Valid values for name are: {coffee_processor_names}
- Use the World Geodetic System (WGS) for the latitude and longitude values.
- When inserting GEOGRAPHY data types use the BigQuery function ST_GEOGPOINT.
- Escape single quotes with a backslash.  Example: Adam's Answer: Adam\'s

Example 1: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Sample'),(2, 'Sample');
Example 2: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Data'),(2, 'Data'),(3, 'Data');

Schema: {schema}
"""

llm_valid_execution = False
while llm_valid_execution == False:
  try:
    sql = LLM(prompt, False, max_output_tokens=1024, temperature=1, top_p=1, top_k=40)
    print("---------------------------------")
    print("sql: ", sql)
    print("---------------------------------")
    llm_valid_execution = RunQuery(sql)
  except Exception as error:
    print("An error occurred:", error)

## Coffee Roaster Table

In [ ]:
coffee_roaster_count = 10

table_name = "coffee_roaster"
primary_key = "coffee_roaster_id"

schema = GetTableSchema(DATASET_ID, table_name)

In [ ]:
table_name = "coffee_roaster"
field_name = "name"
existing_values = GetDistinctValues(DATASET_ID, table_name, field_name)

prompt = f"""Generate {coffee_roaster_count} creative names and return in the below json format.
- The name should be new and not a name that is already used by an existing coffee company.
- The name should be related to coffee roasters that roast the beans to perfection.
- Do not use the following values: {existing_values}

JSON format: [ "value" ]
Sample JSON Response: [ "value1", "value2" ]
"""

llm_valid_execution = False
while llm_valid_execution == False:
  try:
    coffee_roaster_names = LLM(prompt, True, max_output_tokens=1024, temperature=1, top_p=1, top_k=40)
    llm_valid_execution = True
  except Exception as error:
    print("An error occurred:", error)

print(f"coffee_roaster_names: {coffee_roaster_names}")

In [ ]:
starting_value = GetStartingValue(DATASET_ID, table_name, primary_key)

prompt=f"""
You are a database engineer and need to generate data for a table for the below schema.
- The schema is for a Google Cloud BigQuery Table.
- The table name is "{PROJECT_ID}.{DATASET_ID}.{table_name}".
- Read the description of each field for valid values.
- Do not preface the response with any special characters or 'sql'.
- Generate a single INSERT statement with {coffee_roaster_count} rows of data.
- The starting value of the field {primary_key} is {starting_value}.
- Valid values for name are: {coffee_processor_names}
- Use the World Geodetic System (WGS) for the latitude and longitude values.
- When inserting GEOGRAPHY data types use the BigQuery function ST_GEOGPOINT.
- Escape single quotes with a backslash.  Example: Adam's Answer: Adam\'s

Example 1: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Sample'),(2, 'Sample');
Example 2: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Data'),(2, 'Data'),(3, 'Data');

Schema: {schema}
"""

llm_valid_execution = False
while llm_valid_execution == False:
  try:
    sql = LLM(prompt, False, max_output_tokens=1024, temperature=1, top_p=1, top_k=40)
    print("---------------------------------")
    print("sql: ", sql)
    print("---------------------------------")
    llm_valid_execution = RunQuery(sql)
  except Exception as error:
    print("An error occurred:", error)

# Set Locations (by hand)

In [ ]:
sql = f"""ALTER TABLE `{PROJECT_ID}.{DATASET_ID}.location`
  ADD COLUMN IF NOT EXISTS city_id INT OPTIONS(description="The primary city for a location.  For coffee trucks this is the current city."),
  ADD COLUMN IF NOT EXISTS current_latitude FLOAT64 OPTIONS(description="The current latitude of the coffee truck or physical store."),
  ADD COLUMN IF NOT EXISTS current_longitude FLOAT64 OPTIONS(description="The current longitude of the coffee truck or physical store."),
  ADD COLUMN IF NOT EXISTS current_lat_long GEOGRAPHY  OPTIONS(description="The current latitude and longitude of the coffee truck or physical store.")
  ;"""

RunQuery(sql)

In [ ]:
sql = f"""
CREATE TABLE IF NOT EXISTS `{PROJECT_ID}.{DATASET_ID}.location_history`
(
    location_history_id INTEGER NOT NULL OPTIONS(description="Primary key."),
    location_id INTEGER NOT NULL OPTIONS(description="Foreign key: Location table."),
    city_id STRING NOT NULL OPTIONS(description="Foreign key: City table."),
    start_datetime TIMESTAMP NOT NULL OPTIONS(description="The start time of when this location was established"),
    stop_datetime  TIMESTAMP NOT NULL OPTIONS(description="The end time of when this location was moved from."),
    latitude FLOAT64 NOT NULL OPTIONS(description="The latitude of the coffee truck."),
    longitude FLOAT64 NOT NULL OPTIONS(description="The longitude of the coffee truck."),
    lat_long GEOGRAPHY NOT NULL OPTIONS(description="The latitude and longitude of the coffee truck.")
)
CLUSTER BY location_id;
"""

RunQuery(sql)

# Learnings

Issues:
- LLMs are slow
- Scaling to millions of rows is an issue
- Output contains: ```
- Output contains: ```sql
- dates are not always valid dates "2017-09-31" to type DATE
- for UUIDs, use an INT and then swap to UUID later (add a column and then do an update)
- LLM returns single quotes in strings.  Had to prompt or string.replace.
- Probally need to use Min/Max of primary key for ints.
- Sometimes the LLM generates multiple insert..intos
- Inserts are sometimes many INSERTS and not many values

Learnings
- The LLM can generate Small, Medium and Large (3 records for menu items) with pricing that is correct.
- The LLM can understand the schema
- The LLM can understand the description (valid values)
-

**Coffee Tables**
- Company (done)
- Location (done)
- Customer (done, needs loop)
- Customer Profile (created table. needs code to populate LLM items)
- Customer Survey (done, needs loop)
- Customer Review
- Menu
- Order
- Order Detail
- Coffee Farm
- Coffee Processor
- Coffee Exporter
- Coffee Roaster
- Coffee Supply Chain

**Streaming Data**
- Coffee Sales
- Customer Feedback
- Location Telemetry


